# Spotify Suggestion Model V3

## Infrastructure

In [2]:
import sys
!{sys.executable} -m pip install spotipy
# !{sys.executable} -m pip install tensorflow
# !{sys.executable} -m pip install pandas numpy scikit-learn keras scikeras[tensorflow]

In [3]:
username = "agwx2"
cid = "17c11cf9271f4980a85e6aab83cba6bb" 
secret = "015c9abf126340a7981e7ba64dbe7828"
# good_playlist_id = "61BmnG6d7ifmj8Dr5tuCBK"
# bad_playlist_ids = "53FPslFSIAGvVyERcNG4J3"
# source_playlist_id = "5ghFHvZmV4FDK5YLeZQRu9"
good_playlist_ids = [
    "37i9dQZF1DX48TTZL62Yht",   # Hip Hop Favourites
    "37i9dQZF1EQnqst5TRi17F",   # Hip Hop Mix
    "37i9dQZF1DX7FY5ma9162x",   # R&B Favourites
    "37i9dQZF1EQoqCH7BwIYb7",   # R&B Mix
    "53FPslFSIAGvVyERcNG4J3",   # Trap/R&B (personal mix)
    # "37i9dQZF1DWWEcRhUVtL8n"    # Indie Pop
]
bad_playlist_ids = [
    "37i9dQZF1EQpj7X7UK8OOF",   # Rock Mix
    "37i9dQZF1DX1kCIzMYtzum",   # EDM
    "37i9dQZF1DXbITWG1ZJKYt",   # Jazz Classics
    "37i9dQZF1EQmPV0vrce2QZ",   # Country Mix
    "37i9dQZF1EQncLwOalG3K7",   # Pop Mix
    "61BmnG6d7ifmj8Dr5tuCBK"    # Classic Rock (personal mix)
]
source_playlist_ids = [
    # "37i9dQZF1DXb57FjYWz00c",   # 80s Hits
    # "37i9dQZF1EQn2GRFTFMl2A",   # 90s Mix
    # "37i9dQZF1DWUZv12GM5cFk",   # 2000s Hits
    # "37i9dQZF1DXc6IFF23C9jj",   # 2010s Hits
    "5ghFHvZmV4FDK5YLeZQRu9",   # Top 100 (personal mix)
]
source_playlist_id = source_playlist_ids[0]

In [4]:
import time

import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression

from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

In [5]:
def cross_val_analysis(model, x_train, y_train, k = 10, output = True):
    kfold = StratifiedKFold(n_splits=k, shuffle=True)
    results = cross_val_score(model, x_train, y_train, cv=kfold)
    if output:
        print("Cross-Validation Score Standardized: %.2f%% (%.2f%%)" %
            (results.mean() * 100, results.std() * 100))
    return (results.mean() * 100)


## Data Collection

In [6]:
sp_cred = SpotifyClientCredentials(
    client_id=cid,
    client_secret=secret
)
sp = spotipy.Spotify(client_credentials_manager=sp_cred)

In [7]:
all_good_ids = []
for good_playlist_id in good_playlist_ids:
  good_playlist = sp.user_playlist(username, good_playlist_id)
  good_tracks = good_playlist["tracks"]
  good_songs = good_tracks["items"]

  while good_tracks['next']:
      good_tracks = sp.next(good_tracks)
      for item in good_tracks["items"]:
          good_songs.append(item)

  good_ids = []
  for i in range(len(good_songs)):
      good_ids.append(good_songs[i]['track']['id'])
  print(len(good_ids))
  all_good_ids += good_ids
print("")
print(len(all_good_ids))
print(all_good_ids[0])

100
50
100
50
217

517
2BcMwX1MPV6ZHP4tUT9uq6


In [8]:
all_bad_ids = []
for bad_playlist_id in bad_playlist_ids:
  bad_playlist = sp.user_playlist(username, bad_playlist_id)
  bad_tracks = bad_playlist["tracks"]
  bad_songs = bad_tracks["items"]

  while bad_tracks['next']:
      bad_tracks = sp.next(bad_tracks)
      for item in bad_tracks["items"]:
          bad_songs.append(item)

  bad_ids = []
  for i in range(len(bad_songs)):
      bad_ids.append(bad_songs[i]['track']['id'])
  print(len(bad_ids))
  all_bad_ids += bad_ids
print("")
print(len(all_bad_ids))
print(all_bad_ids[0])

50
50
250
50
50
132

582
2WL6GQzPuK9Nrpy9XwNEbz


In [9]:
source_playlist = sp.user_playlist(username, source_playlist_id)
source_tracks = source_playlist["tracks"]
source_songs = source_tracks["items"]

while source_tracks['next']:
    source_tracks = sp.next(source_tracks)
    for item in source_tracks["items"]:
        source_songs.append(item)

source_ids = []
for i in range(len(source_songs)):
    if source_songs[i]['track'] != None:
      source_ids.append(source_songs[i]['track']['id'])
print(len(source_ids))
print(source_ids[0])

147
0ipgd8PPFza3NNmN3Rn2uF


In [10]:
training_features = []

for i in range(0, len(all_good_ids), 50):
  audio_features = sp.audio_features(all_good_ids[i : i + 50])
  for track in audio_features:
    if track != None:

      track['target'] = 1
      training_features.append(track)
      # training_features[-1]['target'] = 1

for i in range(0, len(all_bad_ids), 50):
  audio_features = sp.audio_features(all_bad_ids[i : i + 50])
  for track in audio_features:
    if track != None:
      track['target'] = 0
      training_features.append(track)
      # training_features[-1]['target'] = 0

print(len(training_features))
print(training_features[0])

1099
{'danceability': 0.849, 'energy': 0.424, 'key': 5, 'loudness': -9.579, 'mode': 0, 'speechiness': 0.324, 'acousticness': 0.0635, 'instrumentalness': 0, 'liveness': 0.0834, 'valence': 0.153, 'tempo': 145.887, 'type': 'audio_features', 'id': '2BcMwX1MPV6ZHP4tUT9uq6', 'uri': 'spotify:track:2BcMwX1MPV6ZHP4tUT9uq6', 'track_href': 'https://api.spotify.com/v1/tracks/2BcMwX1MPV6ZHP4tUT9uq6', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2BcMwX1MPV6ZHP4tUT9uq6', 'duration_ms': 242966, 'time_signature': 4, 'target': 1}


In [11]:
source_features = []
for i in range(0, len(source_ids), 50):
    audio_features = sp.audio_features(source_ids[i : i + 50])
    for track in audio_features:
        track['target'] = 0
        source_features.append(track)
        # source_features[-1]['target'] = 0  # arbitrary

print(len(source_features))
print(source_features[0])

147
{'danceability': 0.816, 'energy': 0.754, 'key': 2, 'loudness': -5.922, 'mode': 1, 'speechiness': 0.152, 'acousticness': 0.163, 'instrumentalness': 0.00199, 'liveness': 0.105, 'valence': 0.704, 'tempo': 92.023, 'type': 'audio_features', 'id': '0ipgd8PPFza3NNmN3Rn2uF', 'uri': 'spotify:track:0ipgd8PPFza3NNmN3Rn2uF', 'track_href': 'https://api.spotify.com/v1/tracks/0ipgd8PPFza3NNmN3Rn2uF', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0ipgd8PPFza3NNmN3Rn2uF', 'duration_ms': 125217, 'time_signature': 4, 'target': 0}


## Data Preparation

In [12]:
trainingData = pd.DataFrame(training_features)

In [13]:
trainingData

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,target
0,0.849,0.424,5,-9.579,0,0.3240,0.06350,0.000000,0.0834,0.153,145.887,audio_features,2BcMwX1MPV6ZHP4tUT9uq6,spotify:track:2BcMwX1MPV6ZHP4tUT9uq6,https://api.spotify.com/v1/tracks/2BcMwX1MPV6Z...,https://api.spotify.com/v1/audio-analysis/2BcM...,242966,4,1
1,0.676,0.609,2,-5.831,0,0.4810,0.07060,0.000000,0.1520,0.508,142.037,audio_features,5yY9lUy8nbvjM1Uyo1Uqoc,spotify:track:5yY9lUy8nbvjM1Uyo1Uqoc,https://api.spotify.com/v1/tracks/5yY9lUy8nbvj...,https://api.spotify.com/v1/audio-analysis/5yY9...,237735,4,1
2,0.677,0.557,1,-8.409,1,0.0694,0.00508,0.000008,0.0984,0.100,150.127,audio_features,62Yo3FDddWY8ydu6PW2wyz,spotify:track:62Yo3FDddWY8ydu6PW2wyz,https://api.spotify.com/v1/tracks/62Yo3FDddWY8...,https://api.spotify.com/v1/audio-analysis/62Yo...,206385,4,1
3,0.861,0.656,4,-7.867,0,0.4250,0.10400,0.000000,0.0986,0.331,88.967,audio_features,5W8jRrZ6tWrTrqnKRtIQBf,spotify:track:5W8jRrZ6tWrTrqnKRtIQBf,https://api.spotify.com/v1/tracks/5W8jRrZ6tWrT...,https://api.spotify.com/v1/audio-analysis/5W8j...,218210,4,1
4,0.643,0.403,1,-11.254,0,0.4100,0.34300,0.000000,0.1380,0.111,98.540,audio_features,1fxYz1s15pyZLsJT0Hdh1m,spotify:track:1fxYz1s15pyZLsJT0Hdh1m,https://api.spotify.com/v1/tracks/1fxYz1s15pyZ...,https://api.spotify.com/v1/audio-analysis/1fxY...,174682,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,0.547,0.987,8,-3.146,1,0.0839,0.00191,0.006190,0.3220,0.901,79.976,audio_features,6HjE9DWhhPLSlmIMEyZzRc,spotify:track:6HjE9DWhhPLSlmIMEyZzRc,https://api.spotify.com/v1/tracks/6HjE9DWhhPLS...,https://api.spotify.com/v1/audio-analysis/6HjE...,178627,4,0
1095,0.469,0.537,9,-10.421,1,0.1520,0.02410,0.002010,0.2690,0.783,124.193,audio_features,3aoDEt6zSuYQ47gzarlaVo,spotify:track:3aoDEt6zSuYQ47gzarlaVo,https://api.spotify.com/v1/tracks/3aoDEt6zSuYQ...,https://api.spotify.com/v1/audio-analysis/3aoD...,382834,1,0
1096,0.322,0.265,2,-13.398,1,0.0302,0.07260,0.340000,0.3660,0.207,133.577,audio_features,2gCf9PqIvpKbxwfJMBWwbn,spotify:track:2gCf9PqIvpKbxwfJMBWwbn,https://api.spotify.com/v1/tracks/2gCf9PqIvpKb...,https://api.spotify.com/v1/audio-analysis/2gCf...,226667,4,0
1097,0.373,0.453,9,-9.224,1,0.0419,0.26100,0.000046,0.1600,0.418,95.384,audio_features,4nla8cAvm6AQFnNou9WiR1,spotify:track:4nla8cAvm6AQFnNou9WiR1,https://api.spotify.com/v1/tracks/4nla8cAvm6AQ...,https://api.spotify.com/v1/audio-analysis/4nla...,196841,4,0


In [14]:
train, test = train_test_split(trainingData, test_size = 0.15)

In [15]:
target_features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness", "duration_ms"]

x_total = trainingData[target_features]
y_total = trainingData["target"]
x_train = train[target_features]
y_train = train["target"]
x_test = test[target_features]
y_test = test["target"]

In [16]:
x_train

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
33,0.928,-10.196,0.7450,0.341,0.000000,0.0614,11,0.1030,170233
560,0.744,-7.719,0.6600,0.502,0.000228,0.1830,5,0.0359,348227
181,0.712,-6.174,0.4520,0.703,0.000000,0.0287,4,0.1990,202247
46,0.694,-2.066,0.6900,0.708,0.000000,0.2460,0,0.1370,131587
509,0.841,-7.410,0.0978,0.479,0.000073,0.5450,6,0.0925,175409
...,...,...,...,...,...,...,...,...,...
860,0.460,-7.153,0.5150,0.633,0.000000,0.6130,5,0.0404,503787
380,0.604,-5.979,0.2870,0.673,0.000033,0.0268,1,0.0545,197573
995,0.453,-9.848,0.6410,0.731,0.003950,0.0557,5,0.0345,420067
379,0.705,-6.477,0.3770,0.784,0.000000,0.0133,2,0.2710,264853


In [17]:
y_train

33     1
560    0
181    1
46     1
509    1
      ..
860    0
380    1
995    0
379    1
474    1
Name: target, Length: 934, dtype: int64

In [18]:
x_test

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
748,0.277,-18.442,0.285,0.242,0.464000,0.69400,4,0.0342,824133
64,0.850,-8.032,0.424,0.505,0.009830,0.00220,1,0.0625,164467
175,0.723,-5.926,0.618,0.574,0.000000,0.00970,2,0.0372,194667
417,0.775,-6.839,0.176,0.494,0.000000,0.03890,8,0.2590,184443
480,0.713,-4.513,0.206,0.620,0.000001,0.17100,0,0.0422,211785
...,...,...,...,...,...,...,...,...,...
1011,0.579,-9.484,0.609,0.508,0.000494,0.00574,2,0.0270,391376
485,0.804,-8.018,0.342,0.632,0.000134,0.00186,10,0.0726,217027
183,0.627,-8.529,0.848,0.618,0.000000,0.05580,2,0.0437,197443
639,0.710,-8.948,0.580,0.318,0.000380,0.86700,0,0.0432,194240


In [19]:
y_test

748     0
64      1
175     1
417     1
480     1
       ..
1011    0
485     1
183     1
639     0
915     0
Name: target, Length: 165, dtype: int64

## Model Evaluation

### Random Forest

In [20]:
# random forest combines multiple decision trees and uses bagging
# (training multiple trees on different sections of training data, averaging the result)
rfc = RandomForestClassifier(n_jobs=1, random_state=1)
ts_train_start = time.time()
rfc.fit(x_train, y_train)
ts_train_end = time.time()

ts_inference_start = ts_train_end
rfc_pred = rfc.predict(x_test)
ts_inference_end = time.time()

score = accuracy_score(y_test, rfc_pred) * 100
print("Accuracy using Random Forest: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, rfc_pred))
print("Time Spent: {}sec total, {}sec training, {}sec inference".format(
    round(ts_inference_end - ts_train_start, 3), round(ts_train_end - ts_train_start, 3), round(ts_inference_end - ts_inference_start, 3)))
ts_cv_start = time.time()
cv_score = cross_val_analysis(rfc, x_total, y_total)
ts_cv_end = time.time()
print("Time Spent (Cross-Val): {}sec total".format(round(ts_cv_end - ts_cv_start, 3)))

Accuracy using Random Forest:  87.88 %
Mean squared error: 0.12
Time Spent: 0.12sec total, 0.115sec training, 0.005sec inference
Cross-Validation Score Standardized: 89.63% (1.88%)
Time Spent (Cross-Val): 1.166sec total


Measured Accuracy: 92.12 % / 90.81 %  
Not bad  

### Gradient Boosting

In [21]:
# I <3 gradient boost
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1)
ts_train_start = time.time()
gbc.fit(x_train, y_train)
ts_train_end = time.time()

ts_inference_start = ts_train_end
gbc_pred = gbc.predict(x_test)
ts_inference_end = time.time()

score = accuracy_score(y_test, gbc_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, gbc_pred))
print("Time Spent: {}sec total, {}sec training, {}sec inference".format(
    round(ts_inference_end - ts_train_start, 3), round(ts_train_end - ts_train_start, 3), round(ts_inference_end - ts_inference_start, 3)))
ts_cv_start = time.time()
cv_score = cross_val_analysis(gbc, x_total, y_total)
ts_cv_end = time.time()
print("Time Spent (Cross-Val): {}sec total".format(round(ts_cv_end - ts_cv_start, 3)))

Accuracy using Gradient Boost:  87.27 %
Mean squared error: 0.13
Time Spent: 0.06sec total, 0.059sec training, 0.001sec inference
Cross-Validation Score Standardized: 89.17% (2.80%)
Time Spent (Cross-Val): 0.617sec total


Measured Accuracy: 90.3 % / 90.26 %  
Not bad, slightly worse than Random Forest  

### Gradient Boosting w/ Manual Stacking (Random Forest)

In [22]:
x_train_rfc = (x_train.iloc[0:int(len(x_train) / 2)]).copy()
y_train_rfc = (y_train.iloc[0:int(len(y_train) / 2)]).copy()
x_train_gbc = (x_train.iloc[int(len(x_train) / 2):len(x_train)]).copy()
y_train_gbc = (y_train.iloc[int(len(y_train) / 2):len(y_train)]).copy()

In [23]:
x_test_rfc = x_train_gbc
y_test_rfc = y_train_gbc
x_test_gbc = x_test.copy()
y_test_gbc = y_test.copy()

In [24]:
rfc_stack = RandomForestClassifier(n_jobs=1, random_state=1)
ts_train_start = time.time()
rfc_stack.fit(x_train_rfc, y_train_rfc)
ts_train_end = time.time()

ts_inference_start = ts_train_end
rfc_stack_pred_train = rfc_stack.predict(x_test_rfc)
ts_inference_end = time.time()

score = accuracy_score(y_test_rfc, rfc_stack_pred_train) * 100
print("Accuracy using Random Forest: ", round(score, 2), "%")
print('Mean squared error: %.2f' %
      mean_squared_error(y_test_rfc, rfc_stack_pred_train))
print("Time Spent: {}sec total, {}sec training, {}sec inference".format(
    round(ts_inference_end - ts_train_start, 3), round(ts_train_end - ts_train_start, 3), round(ts_inference_end - ts_inference_start, 3)))
ts_cv_start = time.time()
cv_score = cross_val_analysis(rfc_stack, x_total, y_total)
ts_cv_end = time.time()
print("Time Spent (Cross-Val): {}sec total".format(round(ts_cv_end - ts_cv_start, 3)))

Accuracy using Random Forest:  89.94 %
Mean squared error: 0.10
Time Spent: 0.078sec total, 0.072sec training, 0.006sec inference
Cross-Validation Score Standardized: 89.90% (2.55%)
Time Spent (Cross-Val): 1.164sec total


In [25]:
rfc_stack_pred_test = rfc_stack.predict(x_test_gbc)

In [26]:
# stack
x_train_gbc['rfc_stack_pred'] = rfc_stack_pred_train.tolist()
x_test_gbc['rfc_stack_pred'] = rfc_stack_pred_test.tolist()

In [27]:
x_train_gbc

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms,rfc_stack_pred
567,0.665,-5.931,0.0389,0.738,0.005980,0.01240,0,0.0379,214071,0
1076,0.263,-9.757,0.7140,0.840,0.356000,0.00676,9,0.0527,228267,0
0,0.849,-9.579,0.1530,0.424,0.000000,0.06350,5,0.3240,242966,1
508,0.763,-6.405,0.3490,0.726,0.000000,0.25900,1,0.2820,126346,1
743,0.464,-12.435,0.2820,0.305,0.084600,0.93600,3,0.0316,255227,0
...,...,...,...,...,...,...,...,...,...,...
860,0.460,-7.153,0.5150,0.633,0.000000,0.61300,5,0.0404,503787,0
380,0.604,-5.979,0.2870,0.673,0.000033,0.02680,1,0.0545,197573,1
995,0.453,-9.848,0.6410,0.731,0.003950,0.05570,5,0.0345,420067,0
379,0.705,-6.477,0.3770,0.784,0.000000,0.01330,2,0.2710,264853,1


In [28]:
x_test_gbc

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms,rfc_stack_pred
748,0.277,-18.442,0.285,0.242,0.464000,0.69400,4,0.0342,824133,0
64,0.850,-8.032,0.424,0.505,0.009830,0.00220,1,0.0625,164467,1
175,0.723,-5.926,0.618,0.574,0.000000,0.00970,2,0.0372,194667,1
417,0.775,-6.839,0.176,0.494,0.000000,0.03890,8,0.2590,184443,1
480,0.713,-4.513,0.206,0.620,0.000001,0.17100,0,0.0422,211785,1
...,...,...,...,...,...,...,...,...,...,...
1011,0.579,-9.484,0.609,0.508,0.000494,0.00574,2,0.0270,391376,0
485,0.804,-8.018,0.342,0.632,0.000134,0.00186,10,0.0726,217027,1
183,0.627,-8.529,0.848,0.618,0.000000,0.05580,2,0.0437,197443,0
639,0.710,-8.948,0.580,0.318,0.000380,0.86700,0,0.0432,194240,0


In [29]:
gbc_stack = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1)
ts_train_start = time.time()
gbc_stack.fit(x_train_gbc, y_train_gbc)
ts_train_end = time.time()

ts_inference_start = ts_train_end
gbc_stack_pred = gbc_stack.predict(x_test_gbc)
ts_inference_end = time.time()

score = accuracy_score(y_test_gbc, gbc_stack_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test_gbc, gbc_stack_pred))
print("Time Spent: {}sec total, {}sec training, {}sec inference".format(
    round(ts_inference_end - ts_train_start, 3), round(ts_train_end - ts_train_start, 3), round(ts_inference_end - ts_inference_start, 3)))

Accuracy using Gradient Boost:  87.27 %
Mean squared error: 0.13
Time Spent: 0.033sec total, 0.032sec training, 0.001sec inference


Measured Accuracy: 91.52 %  
In between Gradient Boost and Random Forest, which makes some intuitive sense  

### Stacking Classifier

In [30]:
base_learners = [
  ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
  ('rf_2', GradientBoostingClassifier(n_estimators=100,
   learning_rate=.1, max_depth=1, random_state=1))
]

stk = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
ts_train_start = time.time()
stk.fit(x_train, y_train)
ts_train_end = time.time()

ts_inference_start = ts_train_end
stk_pred = stk.predict(x_test)
ts_inference_end = time.time()

score = accuracy_score(y_test, stk_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, stk_pred))
print("Time Spent: {}sec total, {}sec training, {}sec inference".format(
    round(ts_inference_end - ts_train_start, 3), round(ts_train_end - ts_train_start, 3), round(ts_inference_end - ts_inference_start, 3)))
ts_cv_start = time.time()
cv_score = cross_val_analysis(stk, x_total, y_total)
ts_cv_end = time.time()
print("Time Spent (Cross-Val): {}sec total".format(round(ts_cv_end - ts_cv_start, 3)))

Accuracy using Gradient Boost:  88.48 %
Mean squared error: 0.12
Time Spent: 0.373sec total, 0.371sec training, 0.002sec inference
Cross-Validation Score Standardized: 89.08% (3.89%)
Time Spent (Cross-Val): 3.686sec total


Measured Accuracy: 90.91 % / 89.63 %  
Similar to/exact same as manual stacking example, which makes sense

In [32]:
base_learners = [
    ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('rf_2', KNeighborsClassifier(n_neighbors=5)),
]

stk = StackingClassifier(estimators=base_learners,
                         final_estimator=LogisticRegression())
ts_train_start = time.time()
stk.fit(x_train, y_train)
ts_train_end = time.time()

ts_inference_start = ts_train_end
stk_pred = stk.predict(x_test)
ts_inference_end = time.time()

score = accuracy_score(y_test, stk_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, stk_pred))
print("Time Spent: {}sec total, {}sec training, {}sec inference".format(
    round(ts_inference_end - ts_train_start, 3), round(ts_train_end - ts_train_start, 3), round(ts_inference_end - ts_inference_start, 3)))
ts_cv_start = time.time()
cv_score = cross_val_analysis(stk, x_total, y_total)
ts_cv_end = time.time()
print("Time Spent (Cross-Val): {}sec total".format(round(ts_cv_end - ts_cv_start, 3)))

Accuracy using Gradient Boost:  87.88 %
Mean squared error: 0.12
Time Spent: 0.088sec total, 0.086sec training, 0.002sec inference
Cross-Validation Score Standardized: 88.35% (2.53%)
Time Spent (Cross-Val): 0.824sec total


Measured Accuracy: 90.3 % / 88.35 %    
Better than Random Forest alone


In [33]:
base_learners = [
  ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
  ('rf_2', KNeighborsClassifier(n_neighbors=5)),             
  ('rf_3', GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1))
]

stk = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
ts_train_start = time.time()
stk.fit(x_train, y_train)
ts_train_end = time.time()

ts_inference_start = ts_train_end
stk_pred = stk.predict(x_test)
ts_inference_end = time.time()


score = accuracy_score(y_test, stk_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, stk_pred))
print("Time Spent: {}sec total, {}sec training, {}sec inference".format(
    round(ts_inference_end - ts_train_start, 3), round(ts_train_end - ts_train_start, 3), round(ts_inference_end - ts_inference_start, 3)))
ts_cv_start = time.time()
cv_score = cross_val_analysis(stk, x_total, y_total)
ts_cv_end = time.time()
print("Time Spent (Cross-Val): {}sec total".format(round(ts_cv_end - ts_cv_start, 3)))

Accuracy using Gradient Boost:  87.88 %
Mean squared error: 0.12
Time Spent: 0.366sec total, 0.364sec training, 0.002sec inference
Cross-Validation Score Standardized: 88.90% (2.78%)
Time Spent (Cross-Val): 3.751sec total


Measured Accuracy: 90.91 % / 89.81 %  
Not as good as with all 3, but could potentially be better generalized

### Neural Networks

#### Baseline

In [34]:
def keras_nn_baseline_model_small():
  model = Sequential()
  model.add(Dense(9, input_dim=9, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [35]:
def keras_nn_baseline_model_big():
  model = Sequential()
  model.add(Dense(9, input_dim=9, activation='relu'))
  model.add(Dense(18, input_dim=9, activation='relu'))
  model.add(Dense(90, input_dim=18, activation='sigmoid'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [36]:
def keras_nn_baseline_model_shaped():
  model = Sequential()
  model.add(Dense(5, input_dim=9, activation='relu'))
  model.add(Dense(60, input_dim=5, activation='sigmoid'))
  model.add(Dense(5, input_dim=60, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [37]:
keras_nn_baseline_models = [
    keras_nn_baseline_model_small, keras_nn_baseline_model_big, keras_nn_baseline_model_shaped
]
keras_nn_baseline_model_names = [
    "keras_nn_baseline_model_small", "keras_nn_baseline_model_big", "keras_nn_baseline_model_shaped"
]

num_training_epochs = 100  # 200
batch_size = 5

#### Pipeline w/ Normalization

In [37]:
for keras_nn_baseline_model, keras_nn_baseline_model_name in zip(keras_nn_baseline_models, keras_nn_baseline_model_names):
    # build pipeline
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=keras_nn_baseline_model,
                    epochs=num_training_epochs, batch_size=batch_size, verbose=0)))
    keras_pipeline = Pipeline(estimators)
    print('Running Training & Inference for Keras NN Model "{}"'.format(
        keras_nn_baseline_model_name))
    # train model
    ts_train_start = time.time()
    keras_pipeline.fit(x_train, y_train)
    ts_train_end = time.time()
    # inference w model
    ts_inference_start = ts_train_end
    keras_pipeline_pred = keras_pipeline.predict(x_test)
    ts_inference_end = time.time()
    # validation accuracy
    score = accuracy_score(y_test, keras_pipeline_pred) * 100
    print("Accuracy using Keras: ", round(score, 2), "%")
    print('Mean squared error: %.2f' % mean_squared_error(y_test, stk_pred))
    print("Time Spent: {}sec total, {}sec training, {}sec inference".format(
        round(ts_inference_end - ts_train_start, 3), round(ts_train_end - ts_train_start, 3), round(ts_inference_end - ts_inference_start, 3)))
    print(" ")


/var/folders/fj/p935y7bd3cb5yqyrsf3bmdc40000gn/T/ipykernel_60305/2933901061.py:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimators.append(('mlp', KerasClassifier(build_fn=keras_nn_baseline_model,
2022-05-05 00:52:08.269602: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-05 00:52:08.269761: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-05-05 00:52:08.436579: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Running Training & Inference for Keras NN Model "keras_nn_baseline_model_small"
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-05-05 00:52:08.615358: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-05 00:53:01.875919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
/var/folders/fj/p935y7bd3cb5yqyrsf3bmdc40000gn/T/ipykernel_60305/2933901061.py:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimators.append(('mlp', KerasClassifier(build_fn=keras_nn_baseline_model,


Accuracy using Keras:  89.7 %
Mean squared error: 0.08
Time Spent: 53.656sec total, 53.531sec training, 0.126sec inference
 
Running Training & Inference for Keras NN Model "keras_nn_baseline_model_big"


2022-05-05 00:53:02.127031: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


***Accuracy & Timing Data Saved from Training & Inference on M1 Pro 2021 16in (2 Runs)*** 

**keras_nn_baseline_model_small**  
Accuracy using Keras:  89.7 %, 91.52 %  
Mean squared error: 0.09, 0.09  
Time Spent: 78.995sec(78.943sec-tr,0.051sec-inf), 79.888sec(79.834sec-tr,0.054sec-inf)  
&nbsp;  
&nbsp;  
**keras_nn_baseline_model_big**  
Accuracy using Keras:  91.52 %, 90.3 %  
Mean squared error: 0.09, 0.09  
Time Spent: 94.644sec(94.58sec-tr,0.063sec-inf), 95.661sec(95.591sec-tr,0.069sec-inf)  
&nbsp;  
&nbsp;  
**keras_nn_baseline_model_shaped**  
Accuracy using Keras:  90.91 %, 89.09 %  
Mean squared error: 0.09, 0.09  
Time Spent: 94.325sec(94.256sec-tr,0.07sec-inf), 94.198sec(94.131sec-tr,0.067sec-inf)  

In [ ]:
for keras_nn_baseline_model, keras_nn_baseline_model_name in zip(keras_nn_baseline_models, keras_nn_baseline_model_names):
    # build pipeline
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=keras_nn_baseline_model,
                                              epochs=num_training_epochs, batch_size=batch_size, verbose=0)))
    keras_pipeline = Pipeline(estimators)
    print('Running Cross-Validation Analysis for Keras NN Model "{}"'.format(keras_nn_baseline_model_name))
    ts_cv_start = time.time()
    cv_score = cross_val_analysis(keras_pipeline, x_total, y_total)
    ts_cv_end = time.time()
    print("Time Spent (Cross-Val): {}sec total".format(round(ts_cv_end - ts_cv_start, 3)))
    print(" ")

/var/folders/fj/p935y7bd3cb5yqyrsf3bmdc40000gn/T/ipykernel_75456/3623421867.py:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimators.append(('mlp', KerasClassifier(build_fn=keras_nn_baseline_model,


Running Cross-Validation Analysis for Keras NN Model "keras_nn_baseline_model_small"


2022-05-04 15:20:50.605938: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:22:14.839378: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:22:15.124136: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:23:37.631220: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:23:37.888941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:25:01.573027: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:25:01.828442: I tensorflow/core/grappler/optimizers/cust

Cross-Validation Score Standardized: 89.17% (2.61%)
Time Spent (Cross-Val): 816.291sec total
 
Running Cross-Validation Analysis for Keras NN Model "keras_nn_baseline_model_big"


2022-05-04 15:34:27.273813: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:36:04.704249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:36:05.002347: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:37:42.567021: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:37:42.865753: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:39:23.234883: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:39:23.543189: I tensorflow/core/grappler/optimizers/cust

Cross-Validation Score Standardized: 89.36% (2.10%)
Time Spent (Cross-Val): 976.67sec total
 
Running Cross-Validation Analysis for Keras NN Model "keras_nn_baseline_model_shaped"


2022-05-04 15:50:43.585740: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:52:23.861503: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:52:24.195266: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:54:05.810837: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:54:06.129209: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:55:45.832802: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 15:55:46.519344: I tensorflow/core/grappler/optimizers/cust

Cross-Validation Score Standardized: 81.90% (14.71%)
Time Spent (Cross-Val): 978.022sec total
 


2022-05-04 16:07:01.237939: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


**Cross-Validation Scores**  
keras_nn_baseline_model_small: 89.17 % (816.291sec total)  
keras_nn_baseline_model_big: 89.36 % (976.67sec total)  
keras_nn_baseline_model_shaped: 81.90 % (978.022sec total)  

### Neural Networks w/ Stacking

In [38]:
def get_nn_classifier(baseline_model):
  nn_clf = KerasClassifier(build_fn=baseline_model, epochs=num_training_epochs, batch_size=batch_size, verbose=0)
  # nn_clf._estimator_type = "classifier"
  estimators = []
  estimators.append(('standardize', StandardScaler()))
  estimators.append(('mlp', nn_clf))
  pipeline = Pipeline(estimators)
  return pipeline


In [39]:
base_learners = [
  ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
  ('rf_2', KNeighborsClassifier(n_neighbors=5)),             
  ('rf_3', GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1)),
  ('rf_4', get_nn_classifier(keras_nn_baseline_model_shaped))
]

nn_stk = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
ts_train_start = time.time()
nn_stk.fit(x_train, y_train)
ts_train_end = time.time()

ts_inference_start = ts_train_end
nn_stk_pred = nn_stk.predict(x_test)
ts_inference_end = time.time()


score = accuracy_score(y_test, nn_stk_pred) * 100
print("Accuracy using NN & Stacking: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, nn_stk_pred))
print("Time Spent: {}sec total, {}sec training, {}sec inference".format(
    round(ts_inference_end - ts_train_start, 3), round(ts_train_end - ts_train_start, 3), round(ts_inference_end - ts_inference_start, 3)))


/Users/anuv/miniforge3/envs/mlp/lib/python3.8/site-packages/scikeras/wrappers.py:289: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
2022-05-05 19:47:31.824511: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-05 19:47:31.824622: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-05-05 19:47:31.910724: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-05-05 19:47:32.124992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
/Users/anuv/miniforge3/envs/mlp/lib/python3.8/site-packages/scikeras/wrappers.py:289: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
2022-05-05 19:49:12.811720: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-05 19:50:32.599469: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
/Users/anuv/miniforge3/envs/mlp/lib/python3.8/site-packages/scikeras/wrappers.py:289: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
2022-05-05 19:50:32.892349: I tensorflow/core/gra

Accuracy using NN & Stacking:  87.88 %
Mean squared error: 0.12
Time Spent: 492.53sec total, 492.423sec training, 0.106sec inference


Accuracy using NN & Stacking:  90.3 %, 90.91 %  
Mean squared error: 0.10, 0.09  
Time Spent: 471.285sec(471.213sec-tr,0.072sec-inf), 479.528sec(479.419sec-tr,0.109sec-inf)  

In [43]:
base_learners = [
  ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
  ('rf_2', KNeighborsClassifier(n_neighbors=5)),             
  ('rf_3', GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1)),
  ('rf_4', get_nn_classifier(keras_nn_baseline_model_shaped))
]

nn_stk = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())

ts_cv_start = time.time()
cv_score = cross_val_analysis(nn_stk, x_total, y_total)
ts_cv_end = time.time()
print("Time Spent (Cross-Val): {}sec total".format(round(ts_cv_end - ts_cv_start, 3)))


/Users/anuv/miniforge3/envs/mlp/lib/python3.8/site-packages/scikeras/wrappers.py:289: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
2022-05-05 13:17:41.185851: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
/Users/anuv/miniforge3/envs/mlp/lib/python3.8/site-packages/scikeras/wrappers.py:289: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
2022-05-05 13:19:23.273654: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-05 13:20:43.925745: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
/Users/anuv/miniforge3/envs/mlp/lib/python3.8/sit

KeyboardInterrupt: 

Measured Accuracy: 90.06%

## Generate Predictions

In [ ]:
sourceData = pd.DataFrame(source_features)
sourceData = sourceData[target_features]

In [ ]:
sourceData

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
0,0.662,-13.606,0.9610,0.624,0.000000,0.16500,10,0.0417,252560
1,0.640,-8.114,0.5460,0.647,0.000000,0.12400,5,0.1860,197867
2,0.716,-5.313,0.9100,0.823,0.000000,0.13200,5,0.0495,161747
3,0.468,-6.536,0.7620,0.630,0.000000,0.31000,8,0.0843,204507
4,0.397,-4.577,0.5560,0.903,0.000002,0.00575,2,0.0457,201373
...,...,...,...,...,...,...,...,...,...
117,0.519,-9.982,0.4880,0.181,0.000000,0.94400,5,0.0795,82147
118,0.615,-8.615,0.7470,0.592,0.955000,0.04880,0,0.0443,263673
119,0.512,-5.128,0.8380,0.955,0.000013,0.17200,7,0.3540,40227
120,0.807,-9.729,0.9610,0.510,0.115000,0.04100,0,0.0433,279773


In [ ]:
# model = gbc
# model = keras_pipeline
model = nn_stk

In [ ]:
ts_inference_start = time.time()
pred = model.predict(sourceData)
ts_inference_end = time.time()
print("Time Spent Generating Predictions: {}sec".format(round(ts_inference_end - ts_inference_start, 3)))

In [ ]:
source_playlist_songs = source_playlist['tracks']['items']
source_playlist_songs[0]

{'added_at': '2020-10-01T05:24:42Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/agwx2'},
  'href': 'https://api.spotify.com/v1/users/agwx2',
  'id': 'agwx2',
  'type': 'user',
  'uri': 'spotify:user:agwx2'},
 'is_local': False,
 'primary_color': None,
 'track': None,
 'video_thumbnail': None}

In [ ]:
print("ACCEPTED")


i = 0
for prediction in pred:
    if prediction == 1 and i + 1 < len(source_playlist_songs):
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1

ACCEPTED
Song: Beautiful People (feat. Khalid), By: Ed Sheeran
Song: Revolution 909, By: Daft Punk
Song: That's Life, By: Still Woozy
Song: Order More (feat. Starrah), By: G-Eazy
Song: Paper Trail$, By: Joey Bada$$
Song: Baby, By: Brandy
Song: I Like Dat, By: T-Pain
Song: Lost Cause, By: Billie Eilish
Song: STUNNAMAN (with Roddy Ricch & feat. Lil Wayne), By: Birdman
Song: Baddest (feat. Chris Brown & 2 Chainz), By: Yung Bleu
Song: Y U DON'T LOVE ME? (MISS AMERIKKKA), By: Joey Bada$$
Song: Easy Love, By: Lauv
Song: DRUGS, By: Tai Verdes
Song: Loser, By: Beck
Song: Run To The Sun, By: N.E.R.D
Song: Crush, By: Souly Had
Song: Didn't Cha Know, By: Erykah Badu
Song: Red (Brothel Remix), By: Lucy Daydream
Song: Rocky, By: Still Woozy
Song: Butterflies, By: Michael Jackson
Song: Bake Sale (feat. Travis Scott), By: Wiz Khalifa
Song: CASINO, By: Ashton Travis
Song: favorable colours, By: slenderbodies
Song: Outside, By: Weston Estate
Song: Another One Bites The Dust - Remastered 2011, By: Queen

In [ ]:
print("REJECTED")

i = 0
for prediction in pred:
    if prediction == 0 and i + 1 < len(source_playlist_songs):
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i +
                                            1]['track']['artists'][0]['name']
        print("Song: " + song_title + ", By: " + artist_name)
    i = i + 1